# Дипломчик
### Коды Гоппы

In [1]:
from sage.all import *
import random

In [83]:


class utils:
    def random_binary_unimodular_matrix(rows, cols):
        return random_matrix(GF(2), rows, cols, algorithm='unimodular')
    
    def random_permutation_matrix(n):
        return matrix(GF(2), Permutations(n).random_element().to_matrix())


    def random_irreducible_polynomial(R, degree):
        # var = R.random_element(degree=1).variables()[0] # get var of poly ring
        var = R.gen()

        c = 0
        while True: # todo: benchmark this! how fast we get irreducible poly
            res = 1*var^degree + R.random_element(degree=degree-1)
            c = c + 1

            if res.is_irreducible() == True:
                return res

        # print(f'iterations: {c}') 
        
        return None

    def random_binary_vector(length):
        return vector(random_matrix(GF(2), 1, length))

    def random_binary_err_vector(length, weight_bound):
        list = [0]*length

        weight = 0
        for i in range(length):
            if weight == weight_bound:
                break
        
            # rand_elem = GF(2).random_element() # works bad, gets only null-vec or 't' 1s in the head of vec
            
            rand_elem = random.randint(0,1)
            
            list[i] = rand_elem
            weight = weight + rand_elem

        return vector(GF(2), list)

    def solve_linear_eq(A, b):
        #aug = A.augment(b, subdivide=True)
        #return aug.rref()
        return A.solve_left(b)

    def get_syndrome(R, L, g, y):
        if len(L) != len(y):
            raise Exception("len(L) != len(y)")

        var = R.gen()
        syndrome = R(0)
        for i in range(len(y)):
            frac = 0
            if y[i] != 0:
                frac = (var - L[i]) / y[i]
            
            print(f"frac_{i}: {frac}\n")
            syndrome += utils.get_inverse_by_modulus(frac, g)
            print(f"syndrome_{i}: {syndrome}\n")

        # todo: return syndrome mod g(x)

        S = R.quotient(g, 'x')

        print(f"syndrome: {syndrome}")
        # print(f"syndrome quo rem: {syndrome.quo_rem(g)[0]}")
        # print(f"qual: {syndrome == syndrome.quo_rem(g)[0]}")
        # print(f"syndrome S(): {S(syndrome)}")

        # fr = syndrome
        # for i in range(10):
        #     fr = fr / g
        #     print(f"{i+1}: {fr.factor()}\n")
        
        # return syndrome.quo_rem(g)
        # return syndrome.mod(g)

        print(f"synd.mod: {syndrome.mod(g)}")
        print(f"synd.: {S(syndrome)}")
        
        #return S(syndrome)
        return syndrome.mod(g)
        
        

    def get_d(R, s, g): # USEFUL INFORMATION!!: https://groups.google.com/g/sage-support/c/jNLlKyH2KAw
        # S = R.quotient(g, 'x')

        
        #h = S(s)^(-1) # TODO: is_square() not implemented in quotient rings
        h = utils.get_inverse_by_modulus(s, g)
        print(f"h: {h}\n") # TODO: if h == x => sigma = x => stop
        
        d = (h + x).sqrt(name='x').mod(g)
        print(f"d: {d}\n")

        print(f"d^2: {d^2}\n")
        print(f"h + x mod g: {(h + x).mod(g)}\n")

        (a, t, b) = xgcd(R(g.list()), R(d.list())) # todo: use partial xgcd
        print(f"a: {a}\n")
        print(f"b: {b}\n")
        print(f"t: {t}\n")

        print(f"bv mod g: {(R(b) * R(h)).mod(g)}")
        print(f"bt mod g: {(R(b) * R(t)).mod(g)}")



    def get_inverse_by_modulus(poly, mod): # todo: this func works only if poly and mod has no common divisors
        (g, s, t) = xgcd(poly, mod)
        # print(f"XGCD: {(g,s,t)}\n")
        return s
        

In [3]:
F = GF(2^4, 'a')

R.<x> = F[]
RR.<a> = F

p1 = x^4 + x + a
p2 = (x + a^3)

print(f"GCD: {gcd(p2, p1)}\n")
print(utils.get_inverse_by_modulus(p2, p1))

# S = R.quotient(p1, 'x')
# print(S(p2))


print(p1.quo_rem(p2))

# print(R.random_element(degree=1).variables()[0]) # get var of poly ring
# utils.random_irreducible_polynomial(R, 4)


print(p1.mod(p2))

print(p1.quo_rem(p2)[1])


# (d,u,v) = xgcd(p2,p1);
# print(u.mod(p2))


GCD: 1

XGCD: (1, (a^3 + a + 1)*x^3 + (a^2 + a + 1)*x^2 + (a^3 + a^2 + 1)*x + a^3 + 1, a^3 + a + 1)

(a^3 + a + 1)*x^3 + (a^2 + a + 1)*x^2 + (a^3 + a^2 + 1)*x + a^3 + 1
(x^3 + a^3*x^2 + (a^3 + a^2)*x + a^3 + a + 1, a^2 + 1)
a^2 + 1
a^2 + 1


In [84]:
F = GF(2^4, 'a')

print(f"elems of field F:\n")
for i,x in enumerate(F):  print("{} {}".format(f'{i+1}:', x))

R.<x> = F[]
RR.<a> = F


n = 12
t = 2 # todo: understand how t related to p^m, ans: 2 <= t <= (2^m - 1) / m (but why??)

g = utils.random_irreducible_polynomial(R, t)
# g = x^2 + x + a^3

L = [a for a in random.sample(F.list(), n) if g(a) != 0]
print(f"elems of L:\n")
for i,l in enumerate(L):  print("{} {}".format(f'{i+1}:', l))

C = codes.GoppaCode(g, L)

# n, k, t - publicly available parameter
k = C.dimension()

print(f'\nn: {n}\nk: {k}\nt: {t}\n')

print(f'{C}\n')

for i,v in enumerate(C.list()):  print(f"{i+1}: {v}")
print()

# n - размерность векторного пространства над которым строится код, mt + 1 <= n <= 2^m
# the choice n = 2^m is typical when constructing a Goppa code for a McEliece cryptosystem, but
# Bernstein points out that varying the value of n can yield a better security/eﬃciency trade-oﬀ

# k - размерность пространства кода, для кодов Гоппы справедливо: k >= n - mt
# d - минимальное расстояние кода, для кодов Гоппы справедливо: d >= t + 1, для неприводимого бинарного кода Гоппы: d >= 2t + 1

H = C.parity_check_matrix()
G = C.generator_matrix()

print(f'check matrix:\n{H}\n')
print(f'gen matrix: \n{G}\n')

r = len(G.rows())
c = len(G.columns())

S = utils.random_binary_unimodular_matrix(r, r)
print(f'random binary matrix:\n{S}\n')

P = utils.random_permutation_matrix(c)
print(f'random permutation matrix:\n{P}\n')

GG = S * G * P
print(f"G':\n{GG}\n")

# public key: (G', t)
# private key: (g(x), G, S, P)

# bob sends to alice plaintext m:

m = utils.random_binary_vector(k)
print(f'message: {m}\n')

e = utils.random_binary_err_vector(n, t) # todo: wt(e) <= t!!
print(f'err vec: {e}\n')

c = (m * GG) + e
print(f'cipher text: {c}\n')

# alice decrypts received ciphertext c:

# c * P^(-1) = m * S * G + e * P^(-1) = m * S * G + e'
# then use Patterson algo to find e' = e * P^(-1)
# then find c' - e' = m * S * G
# L: m' = mS => solve m'G = c' - e' and thus find m'
# then find m = m'S^(-1)

# TODO: UNDERSAND XGCD IN MULTIPLICATIVE INVERSION!
syndrome = utils.get_syndrome(R, L, g, c)
print(f"syndrome: {syndrome}")

# todo: READ! https://ask.sagemath.org/question/58891/polynomial-becoming-fractionfieldelement/

utils.get_d(R, syndrome, g)

mSS = G.solve_left(c * P.inverse())
print(f"mS':\n{mSS}\n")

decrypted = mSS * S.inverse()
print(f"decrypted message:\n{decrypted}\n")

print(f"are equal: {decrypted == m}")



elems of field F:

1: 0
2: a
3: a^2
4: a^3
5: a + 1
6: a^2 + a
7: a^3 + a^2
8: a^3 + a + 1
9: a^2 + 1
10: a^3 + a
11: a^2 + a + 1
12: a^3 + a^2 + a
13: a^3 + a^2 + a + 1
14: a^3 + a^2 + 1
15: a^3 + 1
16: 1
elems of L:

1: a
2: a^3 + a^2
3: a^2 + a + 1
4: a^3 + a
5: a^3 + a^2 + a + 1
6: a^2 + 1
7: a^2
8: 1
9: a^3 + 1
10: a^3
11: a^3 + a^2 + 1
12: a^3 + a + 1

n: 12
k: 4
t: 2

[12, 4] Goppa code over GF(2)

1: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
2: (1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0)
3: (0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1)
4: (1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1)
5: (0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0)
6: (1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0)
7: (0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1)
8: (1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1)
9: (0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1)
10: (1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1)
11: (0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0)
12: (1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0)
13: (0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1)
14: (1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1)
15: (0, 1, 1, 1, 1, 0, 0, 

ValueError: matrix equation has no solutions